In [1]:
import pathlib
import textwrap
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

![](./Street_1330_2000x.jpg)

In [3]:
from typing import Sequence

from google.cloud import vision


def analyze_image_from_uri(
    image_uri: str,
    feature_types: Sequence,
) -> vision.AnnotateImageResponse:
    client = vision.ImageAnnotatorClient()

    image = vision.Image()
    image.source.image_uri = image_uri
    features = [vision.Feature(type_=feature_type) for feature_type in feature_types]
    request = vision.AnnotateImageRequest(image=image, features=features)

    response = client.annotate_image(request=request)

    return response


def print_labels(response: vision.AnnotateImageResponse):
    print("=" * 80)
    for label in response.label_annotations:
        print(
            f"{label.score:4.0%}",
            f"{label.description:5}",
            sep=" | ",
        )
        

In [4]:
image_uri = "https://www.neoncreations.co.uk/cdn/shop/products/Street_1330_2000x.jpg"
features = [vision.Feature.Type.LABEL_DETECTION]

response = analyze_image_from_uri(image_uri, features)
print_labels(response)

 98% | Red  
 90% | Orange
 88% | Neon 
 85% | Electronic signage
 82% | Neon sign
 71% | Night
 62% | Sign 


In [5]:
def print_text(response: vision.AnnotateImageResponse):
    print("=" * 80)
    for annotation in response.text_annotations:
        vertices = [f"({v.x},{v.y})" for v in annotation.bounding_poly.vertices]
        print(
            f"{repr(annotation.description):42}",
            ",".join(vertices),
            sep=" | ",
        )
        

In [6]:
image_uri = "https://static.easycanvasprints.com/Upload/mkt/PLA/ECP/BAS_SEM_20170824_MetalStretSigns_2Up_Green_Texthere.jpg" 
features = [vision.Feature.Type.TEXT_DETECTION]

response = analyze_image_from_uri(image_uri, features)
print_text(response)

'YOUR\nAVE\nTEXT HERE\nBLVD'               | (76,89),(437,89),(437,323),(76,323)
'YOUR'                                     | (146,139),(344,98),(355,153),(157,194)
'AVE'                                      | (351,96),(385,89),(388,104),(354,111)
'TEXT'                                     | (85,216),(231,241),(222,294),(76,269)
'HERE'                                     | (245,244),(395,270),(386,322),(236,296)
'BLVD'                                     | (401,273),(437,279),(434,293),(399,287)


In [7]:
def detect_language(text: str) -> dict:
    """Detects the text's language."""
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.detect_language(text)

    print(f"Text: {text}")
    print("Confidence: {}".format(result["confidence"]))
    print("Language: {}".format(result["language"]))

    return result

In [8]:
[detect_language(response.text_annotations[i+1].description) for i in range(len(response.text_annotations)-1)]

Text: YOUR
Confidence: 1
Language: en
Text: AVE
Confidence: 0.8893280625343323
Language: en
Text: TEXT
Confidence: 0.98828125
Language: en
Text: HERE
Confidence: 1
Language: en
Text: BLVD
Confidence: 1
Language: en


[{'language': 'en', 'confidence': 1, 'input': 'YOUR'},
 {'language': 'en', 'confidence': 0.8893280625343323, 'input': 'AVE'},
 {'language': 'en', 'confidence': 0.98828125, 'input': 'TEXT'},
 {'language': 'en', 'confidence': 1, 'input': 'HERE'},
 {'language': 'en', 'confidence': 1, 'input': 'BLVD'}]

In [9]:
def translate_text(target: str, text: str) -> dict:
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print("Text: {}".format(result["input"]))
    print("Translation: {}".format(result["translatedText"]))
    print("Detected source language: {}".format(result["detectedSourceLanguage"]))

    return result


In [11]:
response=[translate_text("ko", response.text_annotations[i+1].description) for i in range(len(response.text_annotations)-1)]

Text: YOUR
Translation: 당신의
Detected source language: en
Text: AVE
Translation: 아베
Detected source language: en
Text: TEXT
Translation: 텍스트
Detected source language: en
Text: HERE
Translation: 여기
Detected source language: en
Text: BLVD
Translation: 블버드
Detected source language: en


In [18]:
print([response[i]['translatedText'] for i in range(len(response))])

['당신의', '아베', '텍스트', '여기', '블버드']


In [19]:
translated_text=[response[i]['translatedText'] for i in range(len(response))]

In [20]:
translated_text

['당신의', '아베', '텍스트', '여기', '블버드']

In [26]:
with open('image_translated.txt', 'w') as f:
    for i in range(len(translated_text)):
        f.write(translated_text[i])
        f.write("\n")